# ES6 알아두면 좋을 것들

- ES6 vs ES6+
- ECMA2015 vs ES6
- http://kangax.github.io/compat-table/es6/

## 목차
1. Function-level scope VS Block-level scope
1. Arrow Functions
1. Template literals
1. Parameter (default, rest, spread)
1. Destructuring assignment
1. Property shorthand
1. Promise
1. Symbol
1. Iteration Protocol
1. Generator
1. Class

… module ...
************

In [2]:
//log util
const log = (...p) => {
	console.log(...p.map(o=>(typeof o == 'object')?[...Object.keys(o)].map((k)=>`${k}=${o[k]}`).join(' '):o));
}

## Function-level scope VS Block-level scope
|Function-level|Block-level|
|---|---|
|함수내에서 유효한 범위|코드 블록내에서 유효한 범위|
|var로 정의|const, let으로 정의|
|var 생략 가능|생략 불가 (생략하면 var)|
|중복 선언도 가능| 중복 선언 불가 (const는 중복 할당 불가)|
|변수 호이스팅|Temporal Dead Zone (선언와 초기화 사이)|
|전역변수로 사용하면 전역 객체의 프로퍼티가 됨(window, global)|전역변수도 개념적인 블록에 존재|

In [ ]:
//var {v:}
{
    console.log('A:', v); //호이스팅
    var v = 100;
    console.log('B:', v);
    var v = 200;
    console.log('C:', v); //변수 중복 선언
}

In [ ]:
//let
{
    let val = 10;
    {
        console.log('A:', val); //Temporal Dead Zone
        //let val = 20; //중복 아님.
    }
}

In [ ]:
//let의 정상적 사용
{
    let val = 10
    {
        let val = 20; //중복 아님.
        console.log('A:', val);
    }
    console.log('B:', val);
}

In [ ]:
//var를 for문에서 사용했을 때
{
    var funcs = [];
    for(var i = 0;i < 5; i++) {
        funcs.push(function() {
            log(i*10);
        });
    }
    
    log({i});
    for(var x = 0; x < funcs.length; x++) {
        funcs[x]();
    }
}

In [ ]:
//var를 for문에서 사용했을 때 -> 클로져로 해결
{
    var funcs = [];
    for(var i = 0;i < 5; i++) {
        funcs.push((function(index) {
            return function() {
                log(index * 10);
            };
        })(i));
    }
    
    for(var x = 0; x < funcs.length; x++) {
        funcs[x]();
    }
}

In [ ]:
//let을 for문에서 사용했을 대
{
    let funcs = [];
    for(let xx = 0;xx < 5; xx++) {
        funcs.push(()=>log(xx*10));
    }
    
//     log({xx});
    for(f of funcs) { //of구문
        f();
    }
    //funcs.forEach(f=>f());
}

In [2]:
//전역변수로 정의
var g = 100;
log(global.g);
let lg = 200;
log(global.lg);

100
undefined


### 정리
- 가능한 var는 사용하지 않는다.
- const를 먼저 고려한다.
- 재할당이 꼭 필요한 경우만 let을 사용한다.
*******

## Arrow Functions
- function keyword 대신 => 이용하여 간략히 함수를 선언
- this를 binding하지 않는다. 함수가 실행되는 context를 가리킴 (Lexical this)
- 위의 이유 때문에 method나 prototype 할당, event listener로 사용하지 않는다.
- 한 줄짜리 , {} 생략된 함수는 return도 묵시적으로 된다.
- arguments가 없다. (rest parameter 사용)
- 작성법
    - (x,y) => { ... }
    - x => { ... }
    - () => { ... } // _ =>
    - x => { return x * x; }
    - x => x * x;

In [7]:
// 표현 축약 
{
    let __ = setTimeout(function() {
        log(10);
    }, 1000);
    
    let _ = setTimeout(()=>log(100), 1000);
}

10
100


In [8]:
// 묵시적 return을 이용한 함수 생성
{
    const add = (a, b) => a + b;
    log(add(10, 20));
    
    const cadd = a => b => a + b;
    const add10 = cadd(10);
    log(add10(20));
    log(add10(30));
}

30
30
40


## Template literals
- 작은 따옴표(')나 큰 따옴표(") 대신 backtick(`)을 사용하는 것
- 여러줄 문자열이 허용된다. (공백까지 포함된다.)
- ${표현식}의 표현식 삽입이 가능하다.

In [9]:
//일반 문자열
{
    var js = 'javascript';
    var str =   '첫번째 줄\n' +
                '두번째 줄:' + js + ' 예제입니다.';
    log(str);
}

첫번째 줄
두번째 줄:javascript 예제입니다.


In [10]:
//backtick 문자열
{
    const js = 'javascript';
    const str = `첫번째 줄
두번째 줄:${js} 예제입니다.`;
    log(str);
}

첫번째 줄
두번째 줄:javascript 예제입니다.


### 정리
- 문자열을 template형태로 사용 가능 (sql, html등)
******

## Parameter (default, rest, spread)
- function을 정의하는 편의 기능
- const f = function(**a = 'defaultValue'**) //default parameter value : 파라미터 체크 와 초기화를 간소화
- const f = function(**...a**) //rest parameter : 인수를 함수 내부에서 배열로 받을 수 있다.
- f(**...a**); // spread operator : 배열 or Iterable을 개별요소로 분리.
- rest는 마지막에만 가능 fuction(...b, a);

In [11]:
//default
{
    const parse = (str, radix=2) => parseInt(str, radix);
    
    log(parse('10'));
    log(parse('10', 10));
    
//     fucntion parse(str, radix) {
//         if(!radix) radix = 2;
//         radix = radix || 2;
//     }
}

2
10


In [12]:
// rest, spread
{
    const sum = (...a) => a.reduce((m,v)=>m+v,0);
    
    log(sum(1, 2, 3));
    
    const arr = [1,2,3];
    log(sum(...arr));
    
    log(sum(...arr, 4, 5));
}

6
6
15


In [13]:
// max, min
{
    //1,3,5,7,9 에서 max, min 구하기
    log(Math.max(1,3,5,7,9), Math.min(1,3,5,7,9));
    
    const list = [1,3,5,7,9];
    log(Math.max(...list), Math.min(...list));
}

9 1
9 1


In [14]:
// default를 이용한 필수값 체크
{
    const err = () => {throw new Error();};
    const testError = (v=err()) => v;

    try {
        log(testError(10));
        log(testError());
    }catch(e) {
        console.log('error!!!!!!');
    }
}

10
error!!!!!!


In [16]:
// array concat
{
    const a = [1, 2];
    const b = [3, 4];
    
    console.log('concat:', a.concat(b));
    console.log('spred:', [...a, ...b]);
    
    console.log('a:', a);
    console.log('b:', b);
}

concat: [ 1, 2, 3, 4 ]
spred: [ 1, 2, 3, 4 ]
a: [ 1, 2 ]
b: [ 3, 4 ]


In [ ]:
// array push (1)
{
    const a = [1, 2];
    const b = [3, 4];
    
    a.push(b);
    console.log(a);
}

In [ ]:
// array push (2)
{
    const a = [1, 2];
    const b = [3, 4];
    
    Array.prototype.push.apply(a, b); //a.push(3, 4);
    console.log(a);
}

In [ ]:
// array push (3)
{
    const a = [1, 2];
    const b = [3, 4];
    
    a.push(...b);
    console.log(a);
}

In [ ]:
// array copy (1)
{
    const a = [1, 2, 3];
    const b = a;
    
    a.push(4);
    console.log(a);
    console.log(b);
}

In [ ]:
// array copy (2)
{
    const a = [1, 2, 3];
    const b = a.slice();
    
    a.push(4);
    console.log(a);
    console.log(b);
}

In [ ]:
// array copy (3)
{
    const a = [1, 2, 3];
    const b = [...a];
    
    a.push(4);
    console.log(a);
    console.log(b);
}

In [ ]:
//객체 병합 (1)
{
    const obj0 = {x:1,  y:2};
    const obj1 = {y:10, z:20};
    
    const merged = Object.assign({}, obj0, obj1);
    console.log(merged, obj0, obj1);
}

In [ ]:
//객체 병합 (2)
{
    const obj0 = {x:1,  y:2};
    const obj1 = {y:10, z:20};
    
    const merged = {...obj0, ...obj1};
    console.log(merged, obj0, obj1);
}

### 정리
- 파라메터의 기본값을 세팅할 수 있어 기본값 확인에 대한 코딩이 줄어든다.
- ...문법으로 array, call을 대체할 수 있다.
*********

## Destructuring assignment
- 객체나 배열을 해체해서 할당하는 기능
- ** const [변수배열] = 배열 **
- ** const {변수배열} = 객체 **

In [ ]:
//배열에 대한 해체 할당
{
    const arr = [1, 2, 3];
    
    const [a, b] = arr;
    console.log(a, b);
    
    const [c, ,d] = arr;
    console.log(c, d);
    
    const [e, ...f] = arr;
    console.log(e, f);
}

In [ ]:
//객체에 대한 해체 할당
{
    const obj = {
        name : '길동',
        age: 60
    }
    
    const {name, age, lastName} = obj;
    console.log(name, age, lastName);
}

In [ ]:
//function과 함께
{
    const display = ({name, age:a}) => {
        return `${name}(${a}세)`;
    }
    const obj = {
        name : '길동',
        age: 60
    }
    
    console.log(display(obj));
}

In [ ]:
// 중첨 개체의 경우
{
    const obj = {
        name : '길동',
        address: {
            zipCode: '03068',
            city: 'seoul'
        }
    }
    
    const {address:{city}} = obj;
    console.log(city);
}

### 정리
- vue.js, react등에서 많이 쓰이는 형태
- 모듈 임포트에서도 많이 사용
- 변수 생성 코드를 줄일 수 있다.
********

## Property shorthand
- 변수와 propery 이름이 갔다면 생략 가능하다.
- 메소드 정의시 function을 생략할 수 있다.

In [ ]:
// property 이름 생략
{
    const x = 1, y = 2;
    const obj = {x:x, y:y};
    console.log('obj:', obj);
    
    const o = {x,y};
    console.log('o:', o);
}

In [ ]:
// 메소드 축약 (1)
{
    const obj = {
        name: 'siva6',
        hello: function() {
            console.log('Hello, ', this.name);
        }
    };
    obj.hello();
}

In [ ]:
// 메소드 축약 (2)
{
    const obj = {
        name: 'siva6',
        hello() {
            console.log('Hello, ', this.name);
        }
    };
    obj.hello();
}

### 정리
- 간편하게 객체를 정의할 수있다.
- 모듈에서 export 할때도 property생략이 자주 사용 된다. export {pi, square, Person};
*****

## Promise
- 비동기 프로그램에서 callback hell을 해소하기 위한 방법 중 하나
- 모다드 형태의 객체를 이용 (thenable)
- 상태
 - pending : 초기상태 (fulfilled, rejected 아님)
 - fulfilled : 연산 수행 성공.
 - rejected : 연산 수행 실패.
 - settled : fulfiled or rejected 상태.
- new Promise((resolve, reject) => {});
- method
 - then
 - catch
- class method
 - all
 - race
 - reject
 - resove
- 구현 함수가 할 일
 - Promise를 반환
 - 값이나 undefined를 반환
 - throw
- es8 : async/await

In [ ]:
//기본 사용법
{
    const asyncFunction = p => {
        return new Promise((resolved, rejected) => {
           setTimeout(()=>resolved('>>'+p), 2000); 
        });
    }
    const promise = asyncFunction('hi!');
    promise.then(console.log);
    promise.then(asyncFunction).then(console.log);
    promise.then(v=>v).then(v=>console.log('**', v));
}

In [ ]:
{
    const DATAS = {'test':{name:'홍길동'}, 'test1':{name:'전우치'}};
    
    const getUser = (id, cb) => {
        setTimeout(()=>cb(DATAS[id].name), 2000);   
    };
    
    console.log('>>10>>');
    getUser('test', console.log);
    console.log('>>20>>');
    
    const getPromiseUser = id => {
        return new Promise((resolved, rejected) => {
           getUser(id, resolved); 
        });
    }
    
    const testUser = getPromiseUser('test1');
    console.log('>>30>>');
    testUser.then(console.log);
    console.log('>>40>>');
}

### 정리
- callback hell을 해소할 수 있다.
- async 프로그램을 sync프로그램처러 처리할 수 있다.
******

## Symbol
- 새로 추가된 7번째 immutalbe primitive data type.
    - (Object, String, Number, Boolean, null, undefined)
- Symbol의 값음 애플리케이션 전체에서 유일하다.
- 중복되지 않는 property key로 주로 사용한다.
- 기본으로 제공되는 내장 Symbol이 있다.
    - Symbol.interator : 객체의 기본 Iterator를 반환하는 메소드(for ... of문)
    - Symbol.match, Symbol.replace, Symbol.search. Symbol.split : 정규표현식 관련 메소드(각 match,replace,search,split과 대응)
    - Symbol.hasInstance : instance 여부 확인 (instanceof)
    - Symbol.species : derived 객체를 생성하는데 사용되는 함수.
    - Symbol.toPrimitive : 객체를 기본형 값으로 변환하는 메소드.
- 전역 심볼을 사용하기 위해서는 Symbol.for(), Symbol.keyFor()를 사용한다.

In [ ]:
{
    // Symbol 생성
    let mySymbol = Symbol(); // new Symobl();은 사용할 수 없다.
    let mySymbolDesc = Symbol('description'); //설명 추가
    let mySymbolDesc2= Symbol('description'); //설명 추가2
    
    console.log('mySymbol', mySymbol, typeof mySymbol);
    console.log('mySymbolDesc', mySymbolDesc, typeof mySymbolDesc);
    console.log('mySymbolDesc2', mySymbolDesc2, typeof mySymbolDesc2);
    console.log('mySymbolDesc == mySymbolDesc2', mySymbolDesc == mySymbolDesc2);
}

In [ ]:
{
    // Symbol 사용법 : Symbol은 유일해서 다른 프로퍼티와 충돌하지 않는다.
    const o = {a:456};
    const mySymbol = Symbol('test');
    
    o[mySymbol] = 123;
    console.log('o:', o);
    console.log('o[mySymbol]:', o[mySymbol]);
    
    for(let i in o) {
        console.log('in:', i);
    }
    
    console.log('JSON.stringify(o):', JSON.stringify(o));
}

In [ ]:
{
    // Well-known Symbol
    const o = {
        a:456,
        [Symbol.toPrimitive]() {
            return this.a;
        }
    };

    console.log(o);
    console.log(o + 544);
}

### 정리
- 새로운 type 이다.
- 중복되지 않는 key로 주로 사용된다.
******

## Iteration Protocol
- iterable과 iterator를 정의해서 순환 가능 자료 구조를 표현한다.
- iterable : 순회 가능한 자료 구조. Symbol.iterator 메소드를 구현한다.
- iterator : Symbol.iterator의 반한 객체.
```
    {
        next() {
            return {
                value: ...,
                done: boolean
            };
        }
    }
```
- built-in iterable : Array, String, Map, Set, DOM

In [12]:
{
    //기본 사용법
    const a = [1, 3, 5];
    log(typeof a[Symbol.iterator]());
    const iteratorOfA = a[Symbol.iterator]();
    log(iteratorOfA.next());
    log(iteratorOfA.next());
    log(iteratorOfA.next());
    log(iteratorOfA.next());
}

object
value=1 done=false
value=3 done=false
value=5 done=false
value=undefined done=true


In [18]:
{
    //for of 사용
    // Array
    for(const v of [1,3,5]) {
        log({v});
    }
    
    // String
    for(const v of 'siva6') {
        log({v});
    }
    
    // Map
    for(const [k,v] of new Map([['a', 'A'], ['b', 'B'], ['c', 'C']])) {
        log({k, v});
    }
    
    // Set
    for(const v of new Set([1,2,2,3])) {
        log({v});
    }
}

v=1
v=3
v=5
v=s
v=i
v=v
v=a
v=6
k=a v=A
k=b v=B
k=c v=C
v=1
v=2
v=3


In [25]:
{
    //무한 카운터
    const counter = ()=>{
        let cnt = 0;
        return {
            next() {
                return {
                    value: cnt++,
                    done: false
                }
            }
        }
    };

    const iterator = counter();
    log(iterator.next().value);
    log(iterator.next().value);
    log(iterator.next().value);
}

0
1
2


In [29]:
{
    //genator
    function* counter() {
        let cnt = 0;
        while(true) yield cnt++;
    }
    
    const iterator = counter();
    log(iterator.next().value);
    log(iterator.next().value);
    log(iterator.next().value);
    
    for(const a of counter()) {
        if(a > 10) break;
        log({a});
    }
}

0
1
2
a=0
a=1
a=2
a=3
a=4
a=5
a=6
a=7
a=8
a=9
a=10


### 정리
- js의 순환 구조는 iteration protocol을 사용한다.
- 배열 보다 좀 더 다앙햔 사용법을 제공한다.
- genertor object는 iterator이면서 iterable이다.
******

## Generator
- iterator를 생성하는 함수.
- Generator는 iterable하다.
- Generator를 생성되는 Generator Object는 interator다.
- 표기법은 function* 이고, 내부에 하나 이상의 yield문이 있어야 한다.
- next()에 parameter를 넘기면 yield의 결과값으로 받을 수 있다.

In [5]:
{
    //기본 사용법
    const generator = function* () {
        let index = 0;
        while(index < 5) yield index++;
    }
    
    const cnt = generator();
    console.log(cnt.next());
    
    for(const a of cnt) {
        console.log(a);
    }
}

{ value: 0, done: false }
1
2
3
4


In [14]:
{
    //next에 parameter 사용하기
    const generator = function* () {
        let index = 0;
        while(true) {
            let reset = yield index++;
            if(reset) index = 0;
        }
    }
    
    const cnt = generator();
    console.log(cnt.next().value);
    console.log(cnt.next().value);
    console.log(cnt.next().value);
    console.log(cnt.next(true).value);
    console.log(cnt.next().value);
}

0
1
2
0
1


In [1]:
{
    // return 은?
    const generator = function* (n) {
        const x = yield n;
        const y = yield x + 1;
        const z = yield y + 2;
        return x + y + z;
    }
    
    const cnt = generator(1);
    console.log(cnt.next());
    console.log(cnt.next(10));
    console.log(cnt.next(20));
    console.log(cnt.next(30));
}

{ value: 1, done: false }
{ value: 11, done: false }
{ value: 22, done: false }
{ value: 60, done: true }


### 정리
- Generator를 이용해 interator를 쉽게 만들 수 있다.
***

## Class
- JS는 prototype-base 객체 지향언어이다. (익숙하지 않다.)
- 하지만, class-base 객체 지향을 지원하게 되면서 추가 되었다.
- class, constructor, super, extends 가 존재하다.
- class body에 member(property)를 선언할 수 없다.
- 상속 후 constructor에서 super()를 실행하지 않은면 this를 참조할 수 없다.

In [18]:
{
    //정의
    //class Node {     // class declarations (hosting 제외)
    const Node = class { // class expressions
        constructor(name) { // 생성자 (하나만 존재한다.)
            Object.assign(this, {_name:name,_text:''});
        } // , 가 없다.
        get name() { // getter
            return this._name;
        }
        get text() { // getter
            return this._text;
        }
        set text(text) { // setter
            this._text = text;
        }
        toString() { // method
            return `<${this._name}>${this._text}</${this._name}>`;
        }
        static getNode(name) { // static method
            return new Node(name);
        }
    }
    
    const div = new Node('div'); // constructor를 실행한다.
    console.log(Node === Node.prototype.constructor);
    console.log(div.name);
    console.log(div.toString());
    div.text = '테스트';
    console.log(div.toString());
    
    const h1 = Node.getNode('h1');
    console.log(h1.toString());
}

true
div
<div></div>
<div>테스트</div>
<h1></h1>


In [29]:
{
    //상속
    const Node = class {
        constructor(name) {
            Object.assign(this, {_name:name,_text:''});
        }
        get name() {
            return this._name;
        }
        get text() {
            return this._text;
        }
        set text(text) {
            this._text = text;
        }
        toString() {
            return `<${this._name}>${this._text}</${this._name}>`;
        }
        static getNode(name) {
            return new Node(name);
        }
    }
    
    const Ul = class extends Node {
        constructor() {
            super('ul');
            Object.assign(this, {_list:[]});
        }
        add(e) {
            this._list.push(e);
            this._text = this._list.reduce((m, v)=>m += v.toString(), '');
        }
    }
    
    const ul = new Ul();
    console.log(ul.toString());
    
    for(const e of ['test', '테스트']) {
        const node = new Node('li');
        node.text = e;
        ul.add(node);
    }
    console.log(ul.toString());
}

<ul></ul>
<ul><li>test</li><li>테스트</li></ul>


### 정리
- prototype이 아닌 class를 사용하는 객체지향을 지원한다.
***

In [ ]:
{
    let obj = {
        id:100
    }
    
    for (let o in obj) {
        console.log(o);
    }
    
    let name = 'name';
    obj[name] = () => 'from name';
    
    let firstName = Symbol();
    obj[firstName] = () => 'from symbol';
    
    console.log(obj[name](), obj[firstName]());
    
    for (let o in obj) {
        console.log(o);
    }
}